In [45]:
import sympy as smp

from sympy import symbols, Eq, Function, dsolve, lambdify
import numpy as np


import plotly.graph_objects as go
from plotly.subplots import make_subplots


## 1D Heat Conduction 

Setup: 
  * Ambient environment with temperature $T_{amb}$ and conductivity $k_{amb}$. 
  * Wall with an exterior surface at $x = W_{start}$, and an interior surface at $x = W_{end}$ that has a temperature of $T_{int}$. The wall has a conductivity of $k_{wall}$. 
  * Steady-state 

Equation for steady state heat conduction[^1] with no heat generation 
$\begin{equation}
\frac{d^2T}{dx^2} = 0 
\end{equation}$

Interface boundary condition at $x = W_{start}$
$\begin{equation}
T(x = W_{start}) = T_{amb}
\end{equation}$

$\begin{equation}
-k_{amb} \frac{\partial T_{amb}(x = W_{start})}{\partial x} = -k_{wall} \frac{\partial T_{wall}(x = W_{start})}{\partial x}
\end{equation}$

Fixed temperature boundary condition at $x = W_{end}$:
$\begin{equation}
T(x = W_{end}) = T_{int}
\end{equation}$



[^1]: or heat transfer more generally?

In [99]:
T_amb = 25 # C 
T_int= 23 #C 

W_start = 0 #m 
W_end = 0.3 #m

k_amb = 1 # conductivity units...
k_wall = 1 # conductivity units 

In [97]:
x, W_start, W_end, T_amb, T_int = symbols('x W_start W_end T_amb T_int', real=True, positive=True)
T = symbols('T', cls=smp.Function)

# W_start, W_end, T_amb, T_int = symbols('W_start W_end T_amb T_int', real=True, positive=True, constant=True)

In [82]:
# differential equation 
Tx = T(x).diff(x, 1)
Txx = T(x).diff(x, 2)
Tx


Derivative(T(x), x)

In [100]:
Tx.subs(x, W_end)

Subs(Derivative(T(x), x), x, 0.3)

In [106]:
flux = -20
eqs = [
    Eq(Txx, 0),
    # Eq(Tx.subs(x, W_start), flux),
    # Eq(Tx.subs(x, W_end), flux)
]
# eqs[1]

In [103]:
eqs[1]

Eq(Subs(Derivative(T(x), x), x, 0.3), -20)

In [107]:
Tx = dsolve(eqs, T(x), ics={T(W_start): T_amb, T(W_end): T_int})

ValueError: 
Input to the funcs should be a list of functions.

In [104]:
# gradient at both sides 

Tx = dsolve(eqs, T(x), ics={T(W_start): T_amb, T(W_end): T_int})
Tx_fun = lambdify([x], Tx.rhs)

x_num = np.linspace(W_start,W_end,100)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_num, y=Tx_fun(x_num), mode='markers',))
fig.update_layout(
    xaxis_title="x [m]",
    yaxis_title="Temperature [ºC]",
    title="Fixed temperature boundary conditions, both sides"
)

ValueError: 
Input to the funcs should be a list of functions.

In [72]:
# fixed temperatures at both sides 
Tx = dsolve(Txx, T(x), ics={T(W_start): T_amb, T(W_end): T_int})
Tx_fun = lambdify([x], Tx.rhs)

x_num = np.linspace(W_start,W_end,100)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_num, y=Tx_fun(x_num), mode='markers',))
fig.update_layout(
    xaxis_title="x [m]",
    yaxis_title="Temperature [ºC]",
    title="Fixed temperature boundary conditions, both sides"
)

### sympy example

In [17]:
f, g = symbols("f g", cls=Function)
x = symbols("x")


In [27]:
eqs = [
    Eq(f(x).diff(x), g(x)),
    Eq(g(x).diff(x), f(x))
]

eqs[0]

Eq(Derivative(f(x), x), g(x))

In [28]:
eqs[1]

Eq(Derivative(g(x), x), f(x))

In [34]:
gen = dsolve(eqs, [f(x), g(x)])
gen, len(gen)

([Eq(f(x), -C1*exp(-x) + C2*exp(x)), Eq(g(x), C1*exp(-x) + C2*exp(x))], 2)

In [32]:
gen[0]

Eq(f(x), -C1*exp(-x) + C2*exp(x))

In [33]:
gen[1]

Eq(g(x), C1*exp(-x) + C2*exp(x))

In [36]:
spec = dsolve(eqs, [f(x), g(x)], ics={f(0): 1, g(2): 3})
spec, len(spec)

([Eq(f(x), (1 + 3*exp(2))*exp(x)/(1 + exp(4)) - (-exp(4) + 3*exp(2))*exp(-x)/(1 + exp(4))),
  Eq(g(x), (1 + 3*exp(2))*exp(x)/(1 + exp(4)) + (-exp(4) + 3*exp(2))*exp(-x)/(1 + exp(4)))],
 2)

In [37]:
spec[0]

Eq(f(x), (1 + 3*exp(2))*exp(x)/(1 + exp(4)) - (-exp(4) + 3*exp(2))*exp(-x)/(1 + exp(4)))